# Dataset

In [ ]:
!pip install pandas seaborn tqdm google-play-scraper

In [ ]:
import os
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

## Alguns App de Comida e Alimentacao no Brasil

1. iFodd ⇒ br.com.brainweb.ifood
2. Zé delivery ⇒ com.cerveceriamodelo.modelonow
3. McDonald's ⇒ com.mcdo.mcdonalds
4. Habibi's ⇒ habibs.alphacode.com.br
5. Uber Eats ⇒ com.ubercab.eats
6. Rappi ⇒ com.grability.rappi
7. Burker King ⇒ burgerking.com.br.appandroid
8. Aiqfome ⇒ com.vanuatu.aiqfome

In [ ]:
apps_ids = [
    'br.com.brainweb.ifood',
    'com.cerveceriamodelo.modelonow',
    'com.mcdo.mcdonalds', 'habibs.alphacode.com.br',
    'com.ubercab.eats',
    'com.grability.rappi',
    'burgerking.com.br.appandroid',
    'com.vanuatu.aiqfome'
]

Dados de Cada Aplicativo

In [ ]:
app_infos = []

for ap in tqdm(apps_ids):
    info = app(ap, lang='en', country='us')
    del info['comments']
    app_infos.append(info)

In [ ]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.head()

## Apps Previews

Queremos:
* Conjunto de dados equilibrado — aproximadamente o mesmo número de avaliações para cada pontuação (1–5)
* Uma amostra representativa das avaliações para cada aplicativo

Podemos satisfazer o primeiro requisito usando a opção de pacote de scraping para filtrar a pontuação da avaliação. Para o segundo, classificaremos as avaliações por sua utilidade, que são as avaliações que o Google Play considera mais importantes.

In [ ]:
app_reviews = []

for ap in tqdm(apps_ids):
    for score in list(range(1, 6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='pt',
                country='br',
                sort=sort_order,
                count= 200 if score == 3 else 100,
                filter_score_with=score
            )
            for r in rvs:
                r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
                r['appId'] = ap
            app_reviews.extend(rvs)

In [ ]:
len(app_reviews)

Salvando avaliações em um arquivo CSV

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)

In [ ]:
app_reviews_df.head()

In [ ]:
os.makedirs("../dataset", exist_ok=True)
app_reviews_df.to_csv('../dataset/ifood Dataset.csv', index=None, header=True)